In [1]:
%pylab inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xgboost as xgb

Populating the interactive namespace from numpy and matplotlib


На прошлой неделе был подготовлен датасет, позволяющий получить скор ~19.5 для июня, 20.39 для мая с использованием xgboost и без использования предсказаний arima

Там уже и так достаточно признаков:
- число поездок за прошлый день/неделю
- скользящие суммы по 6 часов 
- дни недели
- синусы/косинусы, соответствующие недельной сезонности (K=7)

Что ещё можно добавить:
- Добавить ещё скользящих сумм.
- Погода (навеяно задачкой про поездки на велосипедах и погоду)
- Праздники

In [2]:
regions_data = pd.read_csv('region_data_rides_with_features.csv')

In [3]:
regions_data.index = regions_data.time

In [4]:
# if (['time','time.1'] in data.columns):
regions_data.drop('time.1', axis = 1, inplace = True)

In [5]:
#Скользящее среднее : число поездок последних 12 часов
tmp=regions_data.groupby(['region'])['count'].rolling(12).sum()
tmp=tmp.reset_index()
tmp.index=tmp.time
regions_data['last_12_hours_sum'] = tmp['count']
regions_data['last_12_hours_mean']=regions_data['last_12_hours_sum']/12

In [6]:
#Скользящее среднее : число поездок последних 12 часов
tmp=regions_data.groupby(['region'])['count'].rolling(24).sum()
tmp=tmp.reset_index()
tmp.index=tmp.time
regions_data['last_24_hours_sum'] = tmp['count']
regions_data['last_24_hours_mean']=regions_data['last_24_hours_sum']/24

In [7]:
# с этими признаками стало хуже
regions_data['4to9_h_ago']=regions_data.groupby(['region'])['count'].shift(3)
regions_data['7to13_h_ago']=regions_data.groupby(['region'])['count'].shift(6)

In [8]:
base_columns = [ u'region',u'week', u'day',u'hour',
                   u'sink1', u'cosk1', u'sink2', u'cosk2', u'sink3', u'cosk3',
                   u'sink4', u'cosk4', u'sink5', u'cosk5', u'sink6', u'cosk6',
                   u'sink7', u'cosk7', 
                   u'1to6_h_ago', u'2to7_h_ago', u'3to8_h_ago',
                   # с этими признаками стало хуже
                   # u'4to9_h_ago', u'7to13_h_ago',
                   u'last_6_hours_mean', u'last_6_hours_sum',
                   u'last_12_hours_sum', u'last_12_hours_mean',
                   u'last_24_hours_sum','last_24_hours_mean'
               ]

In [9]:
# функция получения идентификатора для проведения сабмита
def get_id(x, a):
    return str(x[0])+'_'+str(x[1])[:10]+'_'+str(int(str(x[1])[11:13]))+'_'+str(a)

In [10]:
# функция обучения, получения предсказания и ошибка
def make_prediction(base_columns, train_time_start,train_time_end,test_time_start,test_time_end,
                    error_for_month, submit = False):

    abs_err_sum=0
    df_submit = pd.DataFrame()
    for i in ['y1','y2','y3','y4','y5','y6']:
        columns_to_use = base_columns
        columns_to_use.append(i+'_sh24')
        columns_to_use.append(i+'_sh168')


        X_train=regions_data[columns_to_use][train_time_start:train_time_end]
        Y_train=regions_data[[i]][train_time_start:train_time_end]

        X_test=regions_data[columns_to_use][test_time_start:test_time_end]

        T_train_xgb = xgb.DMatrix(X_train, Y_train)
        
        # оставим параметры по умолчанию: они творят чудеса
        params = {}
        gbm = xgb.train(dtrain = T_train_xgb, params = params)
        Y_pred = gbm.predict(xgb.DMatrix(X_test))

        predictions = pd.DataFrame(regions_data[[i,'region']][test_time_start:test_time_end])
        predictions['pred'] = Y_pred
        predictions.loc[predictions.pred < 0,'pred'] = 0.0
        
        if(submit):          
            predictions['hour'] = X_test.index
            predictions['id'] = predictions[['region', 'hour']].apply(get_id,args=[i[1]], axis=1)
            df_submit = pd.concat([df_submit, predictions[['id', 'pred']]], axis=0)

        abs_err_sum=abs_err_sum + np.sum(np.abs(predictions.pred - predictions[i]))
        print ('Current error:', abs_err_sum)

    print ('Q error for period: ',abs_err_sum / error_for_month)
           
    return predictions, df_submit

In [49]:
# функция обучения, получения предсказания и ошибка
def make_prediction_params(base_columns, train_time_start,train_time_end,test_time_start,test_time_end,
                    error_for_month, submit = False):

    abs_err_sum=0
    df_submit = pd.DataFrame()
    for i in ['y1','y2','y3','y4','y5','y6']:
        columns_to_use = base_columns
        columns_to_use.append(i+'_sh24')
        columns_to_use.append(i+'_sh168')
        X_train=regions_data[columns_to_use][train_time_start:train_time_end]
        Y_train=regions_data[[i]][train_time_start:train_time_end]
        
        X_test=regions_data[columns_to_use][test_time_start:test_time_end]

        xgb_train = xgb.DMatrix(X_train, Y_train)
        
        params={
            'max_depth': 8, 
            'eta': 0.05, 
            'colsample_bytree': 1,
            "min_child_weight": 8, 
            'subsample': 0.8,
            'gamma': 1, # l2-regularization term
            'alpha': 0.01, # l1-regularization term
            'objective': "reg:linear",
            'eval_metric': 'mae',
            'nthread': -1
        }
        evals = [(xgb_train, 'train')]
        bst = xgb.train(params, xgb_train, num_boost_round=200, early_stopping_rounds=10, evals=evals, verbose_eval=False)
        xgb_test = xgb.DMatrix(X_test)
        Y_pred = bst.predict(xgb_test)

        predictions = pd.DataFrame(regions_data[[i,'region']][test_time_start:test_time_end])
        predictions['pred'] = Y_pred
        predictions.loc[predictions.pred < 0,'pred'] = 0.0
        
        if(submit):          
            predictions['hour'] = X_test.index
            predictions['id'] = predictions[['region', 'hour']].apply(get_id,args=[i[1]], axis=1)
            df_submit = pd.concat([df_submit, predictions[['id', 'pred']]], axis=0)

        abs_err_sum=abs_err_sum + np.sum(np.abs(predictions.pred - predictions[i]))
        print ('Current error:', abs_err_sum)

    print ('Q error for period: ',abs_err_sum / error_for_month)
           
    return predictions, df_submit

In [91]:
regions_data.head(3)

,time,region,count,y1,y2,y3,y4,y5,y6,row,...,y6_sh168,last_12_hours_sum,last_12_hours_mean,last_24_hours_sum,last_24_hours_mean,3to7_h_ago,6to8_h_ago,3to9_h_ago,7to13_h_ago,4to9_h_ago
time,,,,,,,,,,,,,,,,,,,,,
2015-08-01 00:00:00,2015-08-01 00:00:00,1075,64,49.0,56.0,21.0,7.0,9.0,11.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-01 01:00:00,2015-08-01 01:00:00,1075,49,56.0,21.0,7.0,9.0,11.0,11.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-01 02:00:00,2015-08-01 02:00:00,1075,56,21.0,7.0,9.0,11.0,11.0,22.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Смотрим на ошибку в мае

In [34]:
train_time_start = '2016-01-02'
train_time_end = '2016-04-30 23:00:00'
test_time_start = '2016-05-01'
test_time_end = '2016-05-31 17:00:00'

In [93]:
regions_data.index = pd.to_datetime(regions_data.index)

In [94]:
predictions, _ = make_prediction(base_columns, train_time_start,train_time_end,test_time_start,test_time_end, 102*739*6)

Current error: 1325003.7180383205
Current error: 2836684.608457327
Current error: 4402076.549955308
Current error: 5993353.3252872825
Current error: 7583161.690085232
Current error: 9177435.819796383
Q error for period:  20.29202999061703


И ипользованием скользящего среднего были получены результаты
- 20.39 : окно 6
- 20.31 : окно 6 и 12
- 20.28 : окно 6, 12 и 24
Дальше продолжать смыслы нет, качество не сильно возрастает

### Добавление погоды.
Погода может существенно повлиять на число поездок. Посмотрим сначала температуру и ветер. Пример получения метеоданных в соседнем ноутбуке

In [39]:
weather_data = pd.read_csv("weather.csv", names = ['time','temperature','wind','rawtime'], index_col = 'time')
weather_data.index = pd.to_datetime(weather_data.index)
weather_data = weather_data[['temperature','wind']]['2015-08-01 00:00:00':'2016-06-30 17:00:00']
weather_data['temperature'] = (weather_data['temperature']).astype(float)
weather_data['wind'] = (weather_data['wind']).astype(float)

In [36]:
#regions_data2 = regions_data

In [41]:
regions_data = regions_data2

In [42]:
merged = pd.merge(regions_data, weather_data, left_index=True, right_index=True)

In [43]:
regions_data = merged

In [47]:
train_time_start = '2015-08-01'
train_time_end = '2016-04-30 23:00:00'
test_time_start = '2016-05-01'
test_time_end = '2016-05-31 17:00:00'

In [48]:
base_columns = [ u'region',u'week', u'day',u'hour',
                   u'sink1', u'cosk1', u'sink2', u'cosk2', u'sink3', u'cosk3',
                   u'sink4', u'cosk4', u'sink5', u'cosk5', u'sink6', u'cosk6',
                   u'sink7', u'cosk7', 
                   u'1to6_h_ago', u'2to7_h_ago', u'3to8_h_ago',
                   u'last_6_hours_mean', u'last_6_hours_sum',
                   u'last_12_hours_sum', u'last_12_hours_mean',
                   u'last_24_hours_sum','last_24_hours_mean',
                   u'temperature', u'wind'
               ]
predictions, _ = make_prediction(base_columns, train_time_start,train_time_end,test_time_start,test_time_end, 102*739*6)

Current error: 1313266.8039200902
Current error: 2818779.5776881576
Current error: 4361857.36230582
Current error: 5921879.743633211
Current error: 7498335.817202985
Current error: 9083766.878186285
Q error for period:  20.084920618275635


Странно, но базовые параметры температуры не помогли, но всё равно оставим.
Попробуем изменить параметры модели

In [50]:
train_time_start = '2016-01-02'
train_time_end = '2016-04-30 23:00:00'
test_time_start = '2016-05-01'
test_time_end = '2016-05-31 17:00:00'

In [ ]:
%%time
base_columns = [ u'region',u'week', u'day',u'hour',
                   u'sink1', u'cosk1', u'sink2', u'cosk2', u'sink3', u'cosk3',
                   u'sink4', u'cosk4', u'sink5', u'cosk5', u'sink6', u'cosk6',
                   u'sink7', u'cosk7', 
                   u'1to6_h_ago', u'2to7_h_ago', u'3to8_h_ago',
                   u'last_6_hours_mean', u'last_6_hours_sum',
                   u'last_12_hours_sum', u'last_12_hours_mean',
                   u'last_24_hours_sum','last_24_hours_mean',
                   u'temperature', u'wind'
               ]
predictions, _ = make_prediction_params(base_columns, train_time_start,train_time_end,test_time_start,test_time_end, 102*739*6)

Current error: 1172490.179292649
Current error: 2549724.648683816
Current error: 

Делаем предсказания для июня.

In [22]:
# июнь
train_time_start = '2015-08-01'
train_time_end = '2016-05-31 22:00:00'
test_time_start = '2016-05-31 23:00:00'
test_time_end = '2016-06-30 17:00:00'

C:\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



20.6100811262
Wall time: 15min


In [44]:
%%time
_, submit_june = make_prediction_params(train_time_start,train_time_end,test_time_start,test_time_end, 102*715*6, True)

20.3926429294
CPU times: user 19min 13s, sys: 2min 24s, total: 21min 37s
Wall time: 11min 54s
